## Predict what color a card is based on it's text in Magic the Gathering

# Install Wizards of the Coast's API wrapper for python

In [ ]:
!pip install mtgsdk

# Import nessesary tools

In [17]:
import numpy as np
import pandas as pd
from mtgsdk import Card
from mtgsdk import Set
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn_pandas import DataFrameMapper, cross_val_score
from sklearn_pandas.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import mean_absolute_error

# Collect Data

In [103]:
def collect_magic_set(mtg_set_code="###"):
    cards = Card.where(set=mtg_set_code).all()
    return cards

def get_text(magic_set):
    """Uses wizards API to build a dictionary of cards for one "magic_set": name, text, and colors"""
    whole_set = []
    for card in magic_set:
        cards = {
            'text' : card.text,
            'colors' : card.colors,
        }
        
        whole_set.append(cards)
    return whole_set

def get_df(mtg_set_alias):
    magic_set = collect_magic_set(mtg_set_alias)
    magic_set_json = get_text(magic_set)
    magic_set_df = pd.DataFrame(magic_set_json)
    return magic_set_df

In [104]:
m10_df = get_df('m10')

In [105]:
# Preprocessing

# vanilla creatures are creatures with no text and just stats and a name
m10_df['vanilla_creature'] = m10_df['text'].apply(lambda x : 1 if x is None else 0)

In [106]:
m10_df = m10_df.replace('\n',' ', regex=True)
m10_df

,colors,text,vanilla_creature
0,[Green],Deathtouch (Any amount of damage this deals to...,0
1,[Black],"{1}{B}, {T}: Target player loses 1 life.",0
2,[Red],Gain control of target creature until end of t...,0
3,[Blue],Flying (This creature can't be blocked except ...,0
4,[White],+1: You gain 2 life. −1: Put a +1/+1 counter o...,0
5,[Blue],{T}: Target creature an opponent controls atta...,0
6,[],"Whenever a player casts a white spell, you may...",0
7,[White],You gain 7 life.,0
8,[Green],{1}{G}: Create a 1/1 green Insect creature token.,0
9,[White],Enchant creature Enchanted creature gets +1/+1...,0


In [5]:
m13_df = get_df('m13')
m15_df = get_df('m15')
m16_df = get_df('m16')
m17_df = get_df('m17')
m20_df = get_df('m20')